In [3]:
import os, math
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split

# === CONFIG ===
repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
srcA = Path(repo_dir) / "datasets" / "stopmotion" / "trainA"
srcB = Path(repo_dir) / "datasets" / "stopmotion" / "trainB"
out_root = Path(repo_dir) / "datasets" / "stopmotion_pix2pix"
out_train = out_root / "train"
out_test  = out_root / "test"
out_train.mkdir(parents=True, exist_ok=True)
out_test.mkdir(parents=True, exist_ok=True)

target_size = 1024     
test_ratio  = 0.10     

def stemset(p):
    return {p.name for p in p.glob("*") if p.suffix.lower() in [".png", ".jpg", ".jpeg"]}

filesA = sorted([p for p in srcA.glob("*") if p.suffix.lower() in [".png", ".jpg", ".jpeg"]])
namesB = stemset(srcB)
pairs = [ (a, srcB / a.name) for a in filesA if a.name in namesB ]

if not pairs:
    raise RuntimeError("No matching filenames between trainA and trainB. Make sure names match 1-to-1.")

train_pairs, test_pairs = train_test_split(pairs, test_size=test_ratio, random_state=42)

def center_square_crop(im: Image.Image) -> Image.Image:
    w, h = im.size
    s = min(w, h)
    left = (w - s) // 2
    top  = (h - s) // 2
    return im.crop((left, top, left + s, top + s))

def prepare_AB(a_path: Path, b_path: Path) -> Image.Image:
    A = Image.open(a_path).convert("RGB")
    B = Image.open(b_path).convert("RGB")
    A = center_square_crop(A).resize((target_size, target_size), Image.BICUBIC)
    B = center_square_crop(B).resize((target_size, target_size), Image.BICUBIC)
    AB = Image.new("RGB", (target_size * 2, target_size))
    AB.paste(A, (0, 0))
    AB.paste(B, (target_size, 0))
    return AB

def export_split(pairs, out_dir: Path):
    for a_path, b_path in pairs:
        AB = prepare_AB(a_path, b_path)
        out_path = out_dir / (a_path.stem + ".png")
        AB.save(out_path)

print(f"Preparing {len(train_pairs)} train pairs and {len(test_pairs)} test pairs...")
export_split(train_pairs, out_train)
export_split(test_pairs, out_test)
print("Done. Aligned dataset created at:", out_root)


Preparing 867 train pairs and 97 test pairs...
Done. Aligned dataset created at: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\stopmotion_pix2pix


In [2]:
pip install scikit-learn


   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.9 MB 2.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ------------------

In [4]:
import os, shlex, subprocess, sys, pathlib

repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = r"""
python train.py
--dataroot ./datasets/stopmotion_pix2pix
--name stopmotion_pix2pix_1024
--model pix2pix
--dataset_mode aligned
--direction AtoB
--input_nc 3
--output_nc 3
--preprocess none
--load_size 1024
--crop_size 1024
--batch_size 1
--gpu_ids 0
--n_epochs 10
--n_epochs_decay 10
--save_epoch_freq 2
--print_freq 50
--display_freq 200
"""

proc = subprocess.run(shlex.split(" ".join(cmd.split())), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\en

In [7]:
import os, subprocess, shlex

# === CONFIG ===
repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
dataroot = os.path.join(repo_dir, "datasets", "stopmotion_pix2pix")  
exp_name = "stopmotion_pix2pix"  
epoch = "latest"  
gpu_ids = "0"     
num_test = "1000" 

os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = f'python test.py --dataroot "{dataroot}" --name {exp_name} --model pix2pix --direction AtoB --epoch {epoch} --num_test {num_test} --gpu_ids {gpu_ids} --results_dir "{os.path.join(repo_dir, "results")}" --load_size 1024 --crop_size 1024'
print("Running:", cmd)

proc = subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
Running: python test.py --dataroot "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\stopmotion_pix2pix" --name stopmotion_pix2pix --model pix2pix --direction AtoB --epoch latest --num_test 1000 --gpu_ids 0 --results_dir "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results" --load_size 1024 --crop_size 1024
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
W0808 19:27:12.337000 31020 site-packages\torch\_logging\_internal.py:1081] [0/0] Profiler function <class 'torch.autograd.profiler.record_function'> will be ignored
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch

In [8]:
import os, subprocess, shlex

os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCH_COMPILE_DISABLE"]  = "1"
os.environ["TORCHDYNAMO_DISABLE"]    = "1"

# === CONFIG ===
repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
dataroot = os.path.join(repo_dir, "datasets", "stopmotion_pix2pix")  
exp_name = "stopmotion_pix2pix" 
epoch = "latest"
gpu_ids = "0"
num_test = "1000"

os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = f'python test.py --dataroot "{dataroot}" --name {exp_name} --model pix2pix --direction AtoB --epoch {epoch} --num_test {num_test} --gpu_ids {gpu_ids} --results_dir "{os.path.join(repo_dir, "results")}" --load_size 1024 --crop_size 1024'
print("Running:", cmd)

proc = subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
Running: python test.py --dataroot "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\stopmotion_pix2pix" --name stopmotion_pix2pix --model pix2pix --direction AtoB --epoch latest --num_test 1000 --gpu_ids 0 --results_dir "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results" --load_size 1024 --crop_size 1024
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 1024                          	[default: 25

In [ ]:
import os, shlex, subprocess

repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = r"""
python train.py
--dataroot ./datasets/stopmotion_pix2pix
--name stopmotion_pix2pix_1024
--model pix2pix
--dataset_mode aligned
--direction AtoB
--input_nc 3
--output_nc 3
--preprocess none
--load_size 1024
--crop_size 1024t
--batch_size 1
--gpu_ids 0
--n_epochs 50
--n_epochs_decay 20
--save_epoch_freq 2
--print_freq 50
--display_freq 200
--continue_train
--epoch_count 21
"""

proc = subprocess.run(shlex.split(" ".join(cmd.split())), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix


In [4]:
import os, subprocess, shlex

os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCH_COMPILE_DISABLE"]  = "1"
os.environ["TORCHDYNAMO_DISABLE"]    = "1"

# === CONFIG ===
repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
dataroot = os.path.join(repo_dir, "datasets", "stopmotion_pix2pix")  
exp_name = "stopmotion_pix2pix_1024"   
epoch = "latest"
gpu_ids = "0"
num_test = "1000"

os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = f'python test.py --dataroot "{dataroot}" --name {exp_name} --model pix2pix --direction AtoB --epoch {epoch} --num_test {num_test} --gpu_ids {gpu_ids} --results_dir "{os.path.join(repo_dir, "results")}" --load_size 1024 --crop_size 1024'
print("Running:", cmd)

proc = subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
Running: python test.py --dataroot "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\stopmotion_pix2pix" --name stopmotion_pix2pix_1024 --model pix2pix --direction AtoB --epoch latest --num_test 1000 --gpu_ids 0 --results_dir "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results" --load_size 1024 --crop_size 1024
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 1024                          	[defaul

In [10]:
import os, shlex, subprocess

repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = r"""
python train.py
--dataroot ./datasets/stopmotion_pix2pix
--name stopmotion_pix2pix_1024
--model pix2pix
--dataset_mode aligned
--direction AtoB
--input_nc 3
--output_nc 3
--netG unet_256
--lambda_L1 100
--preprocess resize_and_crop
--load_size 1088
--crop_size 1024
--batch_size 1
--gpu_ids 0
--continue_train
--epoch_count 71
--lr 0.0001
--n_epochs 20
--n_epochs_decay 60
--save_epoch_freq 2
--print_freq 50
--display_freq 200
"""

proc = subprocess.run(shlex.split(" ".join(cmd.split())), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\en

In [ ]:
import os, shlex, subprocess

repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = r"""
python train.py
--dataroot ./datasets/stopmotion_pix2pix
--name stopmotion_pix2pix_1024
--model pix2pix
--dataset_mode aligned
--direction AtoB
--input_nc 3
--output_nc 3
--netG unet_256
--lambda_L1 100
--preprocess resize_and_crop
--load_size 1088
--crop_size 1024
--batch_size 1
--gpu_ids 0
--continue_train
--epoch_count 81
--lr 0.0001
--n_epochs 90
--n_epochs_decay 70
--save_epoch_freq 2
--print_freq 50
--display_freq 200
"""

proc = subprocess.run(shlex.split(" ".join(cmd.split())), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix


In [1]:
import os, subprocess, shlex

os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCH_COMPILE_DISABLE"]  = "1"
os.environ["TORCHDYNAMO_DISABLE"]    = "1"

# === CONFIG ===
repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
dataroot = os.path.join(repo_dir, "datasets", "stopmotion_pix2pix") 
exp_name = "stopmotion_pix2pix_1024"  
epoch = "latest"
gpu_ids = "0"
num_test = "1000"

os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = f'python test.py --dataroot "{dataroot}" --name {exp_name} --model pix2pix --direction AtoB --epoch {epoch} --num_test {num_test} --gpu_ids {gpu_ids} --results_dir "{os.path.join(repo_dir, "results")}" --load_size 1024 --crop_size 1024'
print("Running:", cmd)

proc = subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
Running: python test.py --dataroot "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\stopmotion_pix2pix" --name stopmotion_pix2pix_1024 --model pix2pix --direction AtoB --epoch latest --num_test 1000 --gpu_ids 0 --results_dir "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results" --load_size 1024 --crop_size 1024
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 1024                          	[defaul

In [5]:
import os, shlex, subprocess

repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = r"""
python train.py
--dataroot ./datasets/stopmotion_pix2pix
--name stopmotion_pix2pix_1024
--model pix2pix
--dataset_mode aligned
--direction AtoB
--input_nc 3
--output_nc 3
--netG unet_256
--lambda_L1 70
--gan_mode lsgan
--preprocess resize_and_crop
--load_size 1088
--crop_size 1024
--batch_size 1
--gpu_ids 0
--continue_train
--epoch_count 151
--lr 0.00002
--n_epochs 100
--n_epochs_decay 90
--save_epoch_freq 5
--print_freq 50
--display_freq 200
"""

proc = subprocess.run(shlex.split(" ".join(cmd.split())), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\yalda\anaconda3\en

In [6]:
import os, subprocess, shlex

os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCH_COMPILE_DISABLE"]  = "1"
os.environ["TORCHDYNAMO_DISABLE"]    = "1"

# === CONFIG ===
repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
dataroot = os.path.join(repo_dir, "datasets", "stopmotion_pix2pix")  
exp_name = "stopmotion_pix2pix_1024"   
epoch = "46"
gpu_ids = "0"
num_test = "1000"

os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = f'python test.py --dataroot "{dataroot}" --name {exp_name} --model pix2pix --direction AtoB --epoch {epoch} --num_test {num_test} --gpu_ids {gpu_ids} --results_dir "{os.path.join(repo_dir, "results")}" --load_size 1024 --crop_size 1024'
print("Running:", cmd)

proc = subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
Running: python test.py --dataroot "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\stopmotion_pix2pix" --name stopmotion_pix2pix_1024 --model pix2pix --direction AtoB --epoch 46 --num_test 1000 --gpu_ids 0 --results_dir "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results" --load_size 1024 --crop_size 1024
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 1024                          	[default: 2

In [7]:
import os, subprocess, shlex

os.environ["TORCHINDUCTOR_DISABLE"] = "1"
os.environ["TORCH_COMPILE_DISABLE"]  = "1"
os.environ["TORCHDYNAMO_DISABLE"]    = "1"

# === CONFIG ===
repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
dataroot = os.path.join(repo_dir, "datasets", "stopmotion_pix2pix")
exp_name = "stopmotion_pix2pix_1024" 
epoch = "190"
gpu_ids = "0"
num_test = "1000"

os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = f'python test.py --dataroot "{dataroot}" --name {exp_name} --model pix2pix --direction AtoB --epoch {epoch} --num_test {num_test} --gpu_ids {gpu_ids} --results_dir "{os.path.join(repo_dir, "results")}" --load_size 1024 --crop_size 1024'
print("Running:", cmd)

proc = subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
Running: python test.py --dataroot "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\datasets\stopmotion_pix2pix" --name stopmotion_pix2pix_1024 --model pix2pix --direction AtoB --epoch 190 --num_test 1000 --gpu_ids 0 --results_dir "C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix\results" --load_size 1024 --crop_size 1024
C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 1024                          	[default: 

In [ ]:
import os, shlex, subprocess

repo_dir = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(repo_dir)
print("CWD:", os.getcwd())

cmd = r"""
python train.py
--dataroot ./datasets/stopmotion_pix2pix
--name stopmotion_pix2pix_1024
--model pix2pix
--dataset_mode aligned
--direction AtoB
--input_nc 3
--output_nc 3
--netG unet_256
--lambda_L1 100
--gan_mode lsgan
--preprocess resize_and_crop
--load_size 1088
--crop_size 1024
--batch_size 1
--gpu_ids 0
--continue_train
--epoch 46
--epoch_count 47
--lr 0.00005
--n_epochs 20
--n_epochs_decay 40
--save_epoch_freq 2
--print_freq 50
--display_freq 200
"""

proc = subprocess.run(shlex.split(" ".join(cmd.split())), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(proc.stdout)


CWD: C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix
